# Using Tensorflow and horovod

In the markdown cells, I'll be showing the code changed needed

In the end, the horovod loop needs to be in a file, I'll add that file to the repo

In [1]:
import tensorflow as tf

2023-02-09 12:00:16.064807: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Horovod imports - 

```python
import tensorflow as tf
import horovod
import horovod.tensorflow as hvd
```


Ideally you should be having a dataset loaded, not downloaded :)

In [4]:
# Dataset
(mnist_images, mnist_labels), _ = \
    tf.keras.datasets.mnist.load_data(path='mnist.npz')

dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
     tf.cast(mnist_labels, tf.int64))
)
dataset = dataset.repeat().shuffle(10000).batch(128)


In [5]:
# Create model
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])
loss = tf.losses.SparseCategoricalCrossentropy()
opt = tf.optimizers.Adam(0.001)
checkpoint_dir = './checkpoints'
checkpoint = tf.train.Checkpoint(model=mnist_model, optimizer=opt)

Horovod - Scale the learning rate according to number of workers

```python
opt = tf.optimizers.Adam(0.001 * hvd.size())
```

In [12]:
# Define training step

@tf.function
def training_step(images, labels, first_batch):
    with tf.GradientTape() as tape:
        probs = mnist_model(images, training=True)
        loss_value = loss(labels, probs)
    #  tape = hvd.DistributedGradientTape(tape)
    grads = tape.gradient(loss_value, mnist_model.trainable_variables)
    opt.apply_gradients(zip(grads, mnist_model.trainable_variables))
    # if first_batch:
    # :
    # :
    # :
    return loss_value

Horovod - Replace gradientTape with DistributedGradientTape

```python
tape = hvd.DistributedGradientTape(tape)
```

Horovod: broadcast initial variable states from rank 0 to all other processes.
This is necessary to ensure consistent initialization of all workers when
training is started with random weights or restored from a checkpoint.

Note: broadcast should be done after the first gradient step to ensure optimizer
initialization.
```py
        if first_batch:
            hvd.broadcast_variables(mnist_model.variables, root_rank=0)
            hvd.broadcast_variables(opt.variables(), root_rank=0)
```

In [13]:
for batch, (images, labels) in enumerate(dataset.take(100)):
    loss_value = training_step(images, labels)
    if batch % 10 == 0:
        print("Step #%d\tLoss: %.6f" % (batch, loss_value))
        #checkpoint.save(checkpoint_dir) So you can save checkpoint :)


Step #0	Loss: 0.240834
Step #10	Loss: 0.127244
Step #20	Loss: 0.179432
Step #30	Loss: 0.222633
Step #40	Loss: 0.245975
Step #50	Loss: 0.230597
Step #60	Loss: 0.215725
Step #70	Loss: 0.139243
Step #80	Loss: 0.161209
Step #90	Loss: 0.174246


Horovod - Scale the dataset based on num workers

```python
for batch, (images, labels) in enumerate(dataset.take(10000 // hvd.size())):
```

Horovod - Checkpoint saving to be only done in rank one

```py
if hvd.rank() == 0:
    checkpoint.save(checkpoint_dir)
```

All this code will have to be wrapped in a main function, with little more boilerplate code
